# Abcd vs abcD


### Modules

In [1]:
#import os
#import numpy as np
import sympy as sp
sp.init_printing(use_latex='mathjax')
#import pandas as pd
#import ast

#from multiprocessing import Pool, cpu_count
#from tqdm.notebook import tqdm

from snappy import *
S = twister.Surface('S_2_1')

## 6_2

In [2]:
md = 'abcD' #'Dcba' #
print(f"{md}") 

M3 = S.bundle(monodromy=md)
print(M3.volume())

L = M3.exterior_to_link()
print(len(L.DT_code()[0]))
#pd = L.PD_code(); print(pd)

hfk = L.knot_floer_homology()
lmd = list(hfk['ranks'].keys())[-1][1]; print(f"{lmd=}")

abcD
4.4008325161
6
lmd=3


In [3]:
md = 'Abcd' #'DbCa''dCBA' #
print(f"{md}") 

M3 = S.bundle(monodromy=md)
print(M3.volume())

L = M3.exterior_to_link()
print(len(L.DT_code()[0]))
#pd = L.PD_code(); print(pd)

hfk = L.knot_floer_homology()
lmd = list(hfk['ranks'].keys())[-1][1]; print(f"{lmd=}")

Abcd
4.4008325161
6
lmd=3


In [4]:
Ms = [S.bundle(monodromy=mc) for mc in ['Abcd', 'abcD']]
print(f"is_isometric?: {Ms[0].is_isometric_to(Ms[1])}")

is_isometric?: True


## With Siegel representation

In [27]:
K = sp.MatrixSymbol('K',2,2) #([[0,-1],[0,0]])
L = sp.MatrixSymbol('L',2,2) #([[1,0],[1,1]])
I = sp.Identity(2)
O = sp.ZeroMatrix(2,2)
J = sp.MatrixSymbol('J',2,2) #([[0,1],[-1,0]])

In [39]:
Ma = sp.BlockMatrix([[L.inv(), O], [O, I]])
Mb = sp.BlockMatrix([[L.transpose(), K], [K, L.transpose()]])
Mc = sp.BlockMatrix([[I, O], [O, L.inv()]])
Md = sp.BlockMatrix([[I, O], [O, L.transpose()]])
display(Ma, Mb, Mc, Md)

MJ = sp.BlockMatrix([[J, O], [O, J]]); display(MJ)

⎡ -1   ⎤
⎢L    𝟘⎥
⎢      ⎥
⎣ 𝟘   𝕀⎦

⎡ T    ⎤
⎢L   K ⎥
⎢      ⎥
⎢     T⎥
⎣K   L ⎦

⎡𝕀   𝟘 ⎤
⎢      ⎥
⎢    -1⎥
⎣𝟘  L  ⎦

⎡𝕀  𝟘 ⎤
⎢     ⎥
⎢    T⎥
⎣𝟘  L ⎦

⎡J  𝟘⎤
⎢    ⎥
⎣𝟘  J⎦

In [44]:
def ev(M):
    return M.subs([
        (L, sp.Matrix([[1,0],[1,1]])),
        (K, sp.Matrix([[0,-1],[0,0]])),
        (J, sp.Matrix([[0,1],[-1,0]])),
    ]).as_explicit()
evMd = ev(Md)
display(evMd, evMd.det())

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  1⎥
⎢          ⎥
⎣0  0  0  1⎦

1

### Checking if it's an element in Sp(4,Z)

In [86]:
def is_Sp(M):
    if type(M) == sp.matrices.expressions.blockmatrix.BlockMatrix:
        return sp.block_collapse(ev(M).transpose()*ev(MJ)*ev(M)) == ev(MJ)
    else:
        return M.transpose()*ev(MJ)*M == ev(MJ)

for M in {Ma, Mb, Mc, Md}:
    if in_Sp(M):
        print(f"{M} is in Sp(4,Z)")

Matrix([
[L.T,   K],
[  K, L.T]]) is in Sp(4,Z)
Matrix([
[I,       0],
[0, L**(-1)]]) is in Sp(4,Z)
Matrix([
[I,   0],
[0, L.T]]) is in Sp(4,Z)
Matrix([
[L**(-1), 0],
[      0, I]]) is in Sp(4,Z)


### The symplectic expressions of dAbc and Dabc

In [99]:
M_Abcd = sp.block_collapse((Ma.inv())*Mb*Mc*Md)
display(M_Abcd)
M_abcD = sp.block_collapse(Ma*Mb*Mc*(Md.inv()))
display(M_abcD)

⎡   T       -1  T⎤
⎢L⋅L   L⋅K⋅L  ⋅L ⎥
⎢                ⎥
⎢       T  -1  T ⎥
⎣ K    L ⋅L  ⋅L  ⎦

⎡                      -1⎤
⎢ -1  T   -1    -1 ⎛ T⎞  ⎥
⎢L  ⋅L   L  ⋅K⋅L  ⋅⎝L ⎠  ⎥
⎢                        ⎥
⎢                    -1  ⎥
⎢          T  -1 ⎛ T⎞    ⎥
⎣  K      L ⋅L  ⋅⎝L ⎠    ⎦

In [105]:
M1 = ev(M_Abcd); display(M1)
M2 = ev(M_abcD);display(M2)

for M in {M1,M2}:
    print(is_Sp(M))

⎡1  1   1   0⎤
⎢            ⎥
⎢1  2   1   0⎥
⎢            ⎥
⎢0  -1  0   1⎥
⎢            ⎥
⎣0  0   -1  0⎦

⎡1   1   1   -2⎤
⎢              ⎥
⎢-1  0   -1  2 ⎥
⎢              ⎥
⎢0   -1  0   1 ⎥
⎢              ⎥
⎣0   0   -1  2 ⎦

True
True


## Jordan canonical form

In [1]:
P1, JM1 = M1.jordan_form()
display(P1, JM1)

NameError: name 'M1' is not defined

In [ ]:
P2, JM2 = M2.jordan_form()
display(P2, JM2)

In [ ]:
JM1 == JM2

In [ ]:
P1_inv = P1.inv()
display(P1_inv)

In [ ]:
Q = (P1_inv*P2).simplify()
display(Q)

## Rational canonical form

### The charactoristic polynomial of M1 (M2)

In [93]:
cp_M1 = M1.charpoly().as_expr()
cp_M2 = M2.charpoly().as_expr()

print(f"cp_M1 == cp_M2 ?   --> {cp_M1==cp_M2}")

display(cp_M1)

cp_M1 == cp_M2 ?   --> True


 4      3      2          
λ  - 3⋅λ  + 3⋅λ  - 3⋅λ + 1

In [94]:
cp_M1.factor()

 4      3      2          
λ  - 3⋅λ  + 3⋅λ  - 3⋅λ + 1

### The rational form of M1 (M2)

In [95]:
C = sp.matrices.expressions.CompanionMatrix(M1.charpoly()).as_explicit()
display(C, C.det(), is_Sp(C))

⎡0  0  0  -1⎤
⎢           ⎥
⎢1  0  0  3 ⎥
⎢           ⎥
⎢0  1  0  -3⎥
⎢           ⎥
⎣0  0  1  3 ⎦

1

False

### basis of the T-cyclic (sub)space

In [106]:
### Abcd (M1)
v = sp.Matrix(4,1,[0,1,0,0])
vecs = [(M1**k)*v for k in range(5)]
#display(vecs, 3*vecs[3]-3*vecs[2]+3*vecs[1]-vecs[0])

P = sp.Matrix(4,4,lambda i,j: vecs[j][i])
display(P, P.det()) #, P.inv())

#(P.inv())*M1*P
P*C*(P.inv()), M1

⎡0  1   2   4 ⎤
⎢             ⎥
⎢1  2   4   8 ⎥
⎢             ⎥
⎢0  -1  -2  -3⎥
⎢             ⎥
⎣0  0   1   2 ⎦

1

⎛⎡1  1   1   0⎤  ⎡1  1   1   0⎤⎞
⎜⎢            ⎥  ⎢            ⎥⎟
⎜⎢1  2   1   0⎥  ⎢1  2   1   0⎥⎟
⎜⎢            ⎥, ⎢            ⎥⎟
⎜⎢0  -1  0   1⎥  ⎢0  -1  0   1⎥⎟
⎜⎢            ⎥  ⎢            ⎥⎟
⎝⎣0  0   -1  0⎦  ⎣0  0   -1  0⎦⎠

In [114]:
### abcD (M2)
v = sp.Matrix(4,1,[1,0,0,0])
vecs = [(M2**k)*v for k in range(5)]
#display(vecs, 3*vecs[3]-3*vecs[2]+3*vecs[1]-vecs[0])

Q = sp.Matrix(4,4,lambda i,j: vecs[j][i])
display(Q, Q.det()) #, Q.inv())

Q*C*(Q.inv()), M2

⎡1  1   0   0 ⎤
⎢             ⎥
⎢0  -1  -1  -1⎥
⎢             ⎥
⎢0  0   1   1 ⎥
⎢             ⎥
⎣0  0   0   -1⎦

1

⎛⎡1   1   1   -2⎤  ⎡1   1   1   -2⎤⎞
⎜⎢              ⎥  ⎢              ⎥⎟
⎜⎢-1  0   -1  2 ⎥  ⎢-1  0   -1  2 ⎥⎟
⎜⎢              ⎥, ⎢              ⎥⎟
⎜⎢0   -1  0   1 ⎥  ⎢0   -1  0   1 ⎥⎟
⎜⎢              ⎥  ⎢              ⎥⎟
⎝⎣0   0   -1  2 ⎦  ⎣0   0   -1  2 ⎦⎠

### The matrix giving conjugacy of M1 with M2

In [115]:
S2 = Q*(P.inv())
display(S2, S2.det(), is_Sp(S2))

X = S2*M1*(S2.inv())
#display(X)
print(f"X==M2 ? --> {X==M2}")

⎡-1  1  1   0 ⎤
⎢             ⎥
⎢-1  0  -1  0 ⎥
⎢             ⎥
⎢-6  3  -2  -3⎥
⎢             ⎥
⎣-4  2  -1  -2⎦

1

True

X==M2 ? --> True


### Another trial

In [16]:
#### dAbc (M1)

e1 = sp.Matrix(4,1,[1,0,0,0])
vecs = [(M1**k)*e1 for k in range(4)]

P = sp.Matrix(4,4,lambda i,j: vecs[j][i])
display(P, P.det()) #, P.inv())
print(P*C*(P.inv()) == M1)

⎡1  1  2   4 ⎤
⎢            ⎥
⎢0  1  3   7 ⎥
⎢            ⎥
⎢0  0  -1  -2⎥
⎢            ⎥
⎣0  0  0   1 ⎦

-1

True


In [17]:
#### Dabc (M2)

e1 = sp.Matrix(4,1,[0,1,1,0])
vecs = [(M2**k)*e1 for k in range(4)]

Q = sp.Matrix(4,4,lambda i,j: vecs[j][i])
display(Q, Q.det()) #, Q.inv())
print(Q*C*(Q.inv()) == M2)

⎡0  2   2   1 ⎤
⎢             ⎥
⎢1  -1  -3  -4⎥
⎢             ⎥
⎢1  0   1   4 ⎥
⎢             ⎥
⎣0  -1  -1  -2⎦

9

True


### The matrix giving conjugacy of M1 with M2

In [18]:
S1 = Q*(P.inv())
display(S1, S1.det())

⎡0  2   4   -5⎤
⎢             ⎥
⎢1  -2  -1  4 ⎥
⎢             ⎥
⎢1  -1  -2  3 ⎥
⎢             ⎥
⎣0  -1  -2  1 ⎦

-9

In [19]:
X = S1*M1*(S1.inv())
display(X)

print(X==M2)

⎡1   1   1   -1⎤
⎢              ⎥
⎢-1  0   -1  1 ⎥
⎢              ⎥
⎢0   -1  1   0 ⎥
⎢              ⎥
⎣0   0   -1  1 ⎦

True


### One more

In [109]:
### dAbc (M1)
e1 = sp.Matrix(4,1,[0,1,0,1])
vecs = [(M1**k)*e1 for k in range(5)]
#display(vecs, 3*vecs[3]-3*vecs[2]+3*vecs[1]-vecs[0])

P = sp.Matrix(4,4,lambda i,j: vecs[j][i])
display(P, P.det()) #, P.inv())

#(P.inv())*M1*P
P*C*(P.inv()), M1

⎡0  1  3   6 ⎤
⎢            ⎥
⎢1  2  5   11⎥
⎢            ⎥
⎢0  0  -2  -5⎥
⎢            ⎥
⎣1  0  0   2 ⎦

1

⎛⎡1  1   1   0⎤  ⎡1  1   1   0⎤⎞
⎜⎢            ⎥  ⎢            ⎥⎟
⎜⎢1  2   1   0⎥  ⎢1  2   1   0⎥⎟
⎜⎢            ⎥, ⎢            ⎥⎟
⎜⎢0  -1  0   1⎥  ⎢0  -1  0   1⎥⎟
⎜⎢            ⎥  ⎢            ⎥⎟
⎝⎣0  0   -1  0⎦  ⎣0  0   -1  0⎦⎠

In [110]:
### Dabc (M2)
e1 = sp.Matrix(4,1,[1,-1,0,0])
vecs = [(M2**k)*e1 for k in range(5)]
#display(vecs, 3*vecs[3]-3*vecs[2]+3*vecs[1]-vecs[0])

Q = sp.Matrix(4,4,lambda i,j: vecs[j][i])
display(Q, Q.det()) #, Q.inv())

Q*C*(Q.inv()), M2

⎡1   0   0   2 ⎤
⎢              ⎥
⎢-1  -1  -1  -3⎥
⎢              ⎥
⎢0   1   1   0 ⎥
⎢              ⎥
⎣0   0   -1  -3⎦

1

⎛⎡1   1   1   -2⎤  ⎡1   1   1   -2⎤⎞
⎜⎢              ⎥  ⎢              ⎥⎟
⎜⎢-1  0   -1  2 ⎥  ⎢-1  0   -1  2 ⎥⎟
⎜⎢              ⎥, ⎢              ⎥⎟
⎜⎢0   -1  0   1 ⎥  ⎢0   -1  0   1 ⎥⎟
⎜⎢              ⎥  ⎢              ⎥⎟
⎝⎣0   0   -1  2 ⎦  ⎣0   0   -1  2 ⎦⎠

### The matrix giving conjugacy of M1 with M2

In [111]:
S3 = Q*(P.inv())
display(S3, S3.det(), in_Sp(S3))

X = S3*M1*(S3.inv())
#display(X)
print(f"X==M2? --> {X==M2}")

⎡0   0  0   1 ⎤
⎢             ⎥
⎢-1  0  -1  -1⎥
⎢             ⎥
⎢-3  2  0   -2⎥
⎢             ⎥
⎣-2  1  0   -1⎦

1

True

X==M2? --> True


In [112]:
S1 == S2, S2 == S3

(False, False)

In [113]:
Y = (S3.inv())*S1
display(Y, Y.det())

M1*Y == Y*M1

⎡1   1   2   1 ⎤
⎢              ⎥
⎢2   3   6   -2⎥
⎢              ⎥
⎢-2  -1  -5  0 ⎥
⎢              ⎥
⎣0   2   4   -5⎦

-9

False